In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
test_set = pd.read_csv("./datasets/titanic/test.csv")
train_set = pd.read_csv("./datasets/titanic/train.csv")

In [3]:
train_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
from sklearn.base import TransformerMixin , BaseEstimator

class DataFrameSelector(TransformerMixin , BaseEstimator):
    def __init__(self,attributes_names):
        self.attributes_names = attributes_names
    def fit(self,X,y=None):
        return self
    def transform(self,X,y=None):
        return X[self.attributes_names]
    

# Inspired from stackoverflow.com/questions/25239958
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)
        

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

num_pipeline=Pipeline([
    ('num',DataFrameSelector(["Age","SibSp","Parch","Fare"])),
    ('imputer',SimpleImputer(strategy="mean"))
])

cat_pipeline=Pipeline([
    ('cat',DataFrameSelector(["Pclass","Sex","Embarked"])),
    ('cat_imputer',MostFrequentImputer()),
    ('encoder',OneHotEncoder(sparse=False))
    ])

In [7]:
from sklearn.pipeline import FeatureUnion
full_pipeline=FeatureUnion(transformer_list=[
    ('num',num_pipeline),
    ('cat',cat_pipeline)
])

In [8]:
train_set_prepared = full_pipeline.fit_transform(train_set)

In [9]:
test_set_prepared = full_pipeline.fit_transform(test_set)

In [10]:
y_train = train_set["Survived"]
train_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
# from sklearn.ensemble import RandomForestClassifier

# forest_clf = RandomForestClassifier(n_estimators=200,random_state=42)
# forest_clf.fit(train_set_prepared,y_train)

In [12]:
# from xgboost import XGBClassifier

# xgbooss_clf = XGBClassifier()
# xgbooss_clf.fit(train_set_prepared,y_train)

In [13]:
def save_titanic_pred(y,test_set):
    prediction = pd.DataFrame(y,columns=["Survived"],index=list(test_set["PassengerId"]))
    prediction.index.names=['PassengerId']
    prediction.to_csv("_prediction1.csv")
    return prediction

In [14]:
# y_test = forest_clf.predict(test_set_prepared)

In [15]:
# y_test_xgboost = xgbooss_clf.predict(test_set_prepared)

In [16]:
# save_titanic_pred(y_test,test_set)

In [17]:
# save_titanic_pred(y_test_xgboost,test_set)


# Voting Classifier

In [18]:
# from sklearn.ensemble import VotingClassifier
# from sklearn.svm import SVC

# forest_clf_vtg=RandomForestClassifier(n_estimators=100, random_state=0,criterion='entropy')
# xgboost_clf_vtg=XGBClassifiaer()
# svc_clf = SVC(probability=True)

# voting_clf= VotingClassifier(
#     estimators=[('fc',forest_clf_vtg),('xg', xgboost_clf_vtg),('svc',svc_clf)],
#     voting='soft')
# voting_clf.fit(train_set_prepared,y_train)



In [19]:
# y_vtg_pred = voting_clf.predict(test_set_prepared) 
# save_titanic_pred(y_vtg_pred , test_set)

# Neural Network

In [20]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
train_set_prepared_std1 = std.fit_transform(train_set_prepared)
train_set_prepared_std1

array([[-5.92480600e-01,  4.32793366e-01, -4.73673609e-01, ...,
        -4.82042680e-01, -3.07562343e-01,  6.15838425e-01],
       [ 6.38789012e-01,  4.32793366e-01, -4.73673609e-01, ...,
         2.07450510e+00, -3.07562343e-01, -1.62380254e+00],
       [-2.84663197e-01, -4.74545196e-01, -4.73673609e-01, ...,
        -4.82042680e-01, -3.07562343e-01,  6.15838425e-01],
       ...,
       [ 4.37434839e-15,  4.32793366e-01,  2.00893337e+00, ...,
        -4.82042680e-01, -3.07562343e-01,  6.15838425e-01],
       [-2.84663197e-01, -4.74545196e-01, -4.73673609e-01, ...,
         2.07450510e+00, -3.07562343e-01, -1.62380254e+00],
       [ 1.77062908e-01, -4.74545196e-01, -4.73673609e-01, ...,
        -4.82042680e-01,  3.25137334e+00, -1.62380254e+00]])

In [21]:
import tensorflow as tf

In [22]:
ann = tf.keras.models.Sequential()

In [23]:
ann.add(tf.keras.layers.Dense(units=128,activation="relu"))
ann.add(tf.keras.layers.Dense(units=64,activation="relu"))
ann.add(tf.keras.layers.Dense(units=32,activation="relu"))
ann.add(tf.keras.layers.Dense(units=16,activation="relu"))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [24]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [25]:
ann.fit(train_set_prepared_std1, y_train, batch_size = 32, epochs = 200)

Epoch 1/200
28/28 [==============================] - 0s 2ms/step - loss: 0.5820 - accuracy: 0.7486
Epoch 2/200
28/28 [==============================] - 0s 1ms/step - loss: 0.4431 - accuracy: 0.8070
Epoch 3/200
28/28 [==============================] - 0s 1ms/step - loss: 0.4159 - accuracy: 0.8182
Epoch 4/200
28/28 [==============================] - 0s 1ms/step - loss: 0.4070 - accuracy: 0.8283
Epoch 5/200
28/28 [==============================] - 0s 1ms/step - loss: 0.3963 - accuracy: 0.8339
Epoch 6/200
28/28 [==============================] - 0s 1ms/step - loss: 0.3930 - accuracy: 0.8339
Epoch 7/200
28/28 [==============================] - 0s 1ms/step - loss: 0.3893 - accuracy: 0.8373
Epoch 8/200
28/28 [==============================] - 0s 1ms/step - loss: 0.3842 - accuracy: 0.8406
Epoch 9/200
28/28 [==============================] - 0s 1ms/step - loss: 0.3778 - accuracy: 0.8406
Epoch 10/200
28/28 [==============================] - 0s 1ms/step - loss: 0.3803 - accuracy: 0.8373
Epoch 11/

28/28 [==============================] - 0s 1ms/step - loss: 0.2794 - accuracy: 0.8822
Epoch 84/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2749 - accuracy: 0.8810
Epoch 85/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2821 - accuracy: 0.8810
Epoch 86/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2771 - accuracy: 0.8844
Epoch 87/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2781 - accuracy: 0.8833
Epoch 88/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2790 - accuracy: 0.8810
Epoch 89/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2863 - accuracy: 0.8743
Epoch 90/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2802 - accuracy: 0.8777
Epoch 91/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2725 - accuracy: 0.8855
Epoch 92/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2719 - accuracy: 0.8822
Epoch 93/200


28/28 [==============================] - 0s 1ms/step - loss: 0.2458 - accuracy: 0.8990
Epoch 165/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2486 - accuracy: 0.8889
Epoch 166/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2534 - accuracy: 0.8866
Epoch 167/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2617 - accuracy: 0.8810
Epoch 168/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2490 - accuracy: 0.8866
Epoch 169/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2464 - accuracy: 0.8866
Epoch 170/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2422 - accuracy: 0.8956
Epoch 171/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2430 - accuracy: 0.8945
Epoch 172/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2431 - accuracy: 0.8878
Epoch 173/200
28/28 [==============================] - 0s 1ms/step - loss: 0.2460 - accuracy: 0.8889
Epoc

In [28]:
def boolstr_to_floatstr(v):
    if v == 'True':
        return '1'
    elif v == 'False':
        return '0'
    else:
        return v

In [30]:
ann_predict1 = ann.predict(std.transform(test_set_prepared))

In [31]:
ann_predict1 = (ann_predict1>0.5)
new_data = np.vectorize(boolstr_to_floatstr)(ann_predict1).astype(int)

In [32]:
new_data

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
    

In [33]:
save_titanic_pred(new_data,test_set)

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,0
...,...
1305,0
1306,1
1307,0
